In [2]:
import pandas as pd
import dill
import re

# opening the saved files 
# which are used to predict the tags 
# to the questions asked by the user

classifier = dill.load(open('model_data.sav', 'rb'))
tfvectorizer = dill.load(open('tfidf_data.sav', 'rb'))
vectorizer = dill.load(open('bow_data.sav', 'rb'))

In [3]:
import re   # importing regular expressions used for cleaning texts

# importing natural language toolkit
# that helps in cleaning texts by using
# stopwords, SnowballStemmer, WordNetLemmatizer libraries

from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
def clean(s):
    s=str(s)
    s=s.lower()
    html=re.compile('<.*?>')   #removing html tags
    cleaned = re.sub(html,' ',s)
    fil=[]
    for i in cleaned.split():     # splits the text and repalces the unwanted characters with ''
        if i!='c++':
            cleaned=re.sub('[^A-Za-z]', '', i)     #search the pattern !(A-Z & a-z) and replace with ''
            fil.append(cleaned)
        else:
            fil.append(i)
    return fil        # returns the splitted text with removed stopwords and html tags


stop=set(stopwords.words('english'))    #loading stopwords in english to compare and remove
sno=SnowballStemmer('english')


# function to stem the data
# stemming means grouping the words
# after cleaning the data for further processing

def stem(s):
    fil=[]
    for _ in s:
        if _ not in stop:
            s=(sno.stem(_).encode('utf8'))   # encoding the data into a clean file
            fil.append(s)
    s=b' '.join(fil)
    return s

In [14]:
from tkinter import *
from PIL import ImageTk, Image
import os
root = Tk()

#####  Title
root.title("Autonomous Tagging Of Stackoverflow Questions")

##### Image of stacoverflow logo
img = ImageTk.PhotoImage(Image.open("index.png"))   
panel = Label(root, image = img)
panel.image = img
panel.grid(row = 0, column = 0)


img1 = ImageTk.PhotoImage(Image.open("st.jpg"))
label1 = Label(root,image = img1)
label1.image = img1
label1.grid(row = 1, column = 0)


label2 = Label(root, text = "Ask Me a Question!!!",font = "Arial 20 bold", fg = 'black')
label2.grid(row = 2, column = 0)

##### Text box to enter the question
ques = StringVar()
quesEntered = Entry(root, width = 40, textvariable = ques,font="Arial 18 bold")
quesEntered.grid(column = 0, row = 3, padx = 3, pady = 3)


# function that takes the question from user
# predicts the tags to the questions
ans = StringVar()
def pred():
    t = ques.get()
    l=[]
    l.append(stem(clean(t)))
    x=tfvectorizer.transform(l)
    t=classifier.predict(x)
    k=vectorizer.inverse_transform(t)
    res = re.sub('[^A-Za-z#+-]+', ' ', str(k[0]))
    ans.set(res)
    label3 = Entry(root,textvariable = ans, font = "Arial 20 bold")
    label3.grid(column =0, row = 5, padx = 3, pady = 3)


##### Button that predicts the tags by calling the function predict
button = Button(root,text = "Predict Tag", font = "Arial 18 bold", command = pred)
button.grid(column= 0, row = 4, padx = 3, pady = 3)



root.mainloop()
